In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=73, Om0=0.25)

import astropy.units as u
from my_functions import *

w_central = central_wavelength()

In [ ]:
%matplotlib inline

In [ ]:
def Get_d_RA_DEC_from_xyz(x_Arr, y_Arr, z_Arr):
    distance = (x_Arr**2 + y_Arr**2 + z_Arr**2)**0.5
    DEC = np.pi/2.0 - np.arccos(z_Arr/distance)
    RA = np.arctan(y_Arr/x_Arr)

    DEC = np.rad2deg(DEC)
    RA = np.rad2deg(RA)
    return distance, RA, DEC

In [ ]:
def M_to_m(M, redshift, x, y, z):
    dL = cosmo.luminosity_distance(redshift).to(u.pc).value.reshape(-1, 1)
    return M + 5 * np.log10(dL) - 5

def line_flux():
    lc = np.load('/home/alberto/almacen/David_lightcone/LightCone_512_15sep2022.npy')
    XX, YY, ZZ = lc['pos'].T * cosmo.h
    z = lc['redshift']
    OII_flux = lc['FluxDust_OII3727'] + lc['FluxDust_OII3729']
    OII_flux_nodust = lc['Flux_OII3727'] + lc['Flux_OII3729']
    Hbeta_flux = lc['FluxDust_Hbeta']
    # M_noline = lc['ObsMagDustNoLine']
    M_line = lc['ObsMagDustLine']
    pm_mag_line = M_to_m(M_line, z, XX, YY, ZZ).T
    pm_flx_line = mag_to_flux(pm_mag_line, w_central.reshape(-1, 1))
    # pm_mag_noline = M_to_m(M_noline, z, XX, YY, ZZ).T
    # pm_flx_noline = mag_to_flux(pm_mag_noline, w_central.reshape(-1, 1))
    dist, ra, dec = Get_d_RA_DEC_from_xyz(XX, YY, ZZ)

    return OII_flux, OII_flux_nodust, Hbeta_flux, z, pm_flx_line, dist, ra, dec
OII_flux, OII_flux_nodust, Hbeta_flux, gal_z, pm_flx_line, dist, ra, dec = line_flux()

dL = cosmo.luminosity_distance(gal_z).to(u.cm).value
L_OII = np.log10(OII_flux * 4 * np.pi * dL ** 2)
L_OII_nodust = np.log10(OII_flux_nodust * 4 * np.pi * dL ** 2)
L_Hbeta = np.log10(Hbeta_flux * 4 * np.pi * dL ** 2)
mag = flux_to_mag(pm_flx_line[-2], w_central[-2])

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))

bins = np.linspace(16, 26, 70)
bins_w = bins[1] - bins[0]

area_obs = 10
ax.hist(mag, bins=bins, histtype='step', linewidth=2,
        weights=np.full(mag.shape, (bins_w * area_obs) ** -1), label='10 deg$^2$')
area_obs = 3
ax.hist(mag, bins=bins, histtype='step', linewidth=2,
        weights=np.full(mag.shape, (bins_w * area_obs) ** -1), label='3 deg$^2$')

ref = pd.read_csv('/home/alberto/Desktop/r_abundance_gal_mock.csv', header=None)
ax.plot(ref[0], ref[1], ls='--', linewidth=2, c='k', label='Izquierdo-Villalba+2019')

ax.set_yscale('log')
ax.set_ylim(1, 1e6)
ax.set_xlim(16, 26)
ax.set_ylabel('dN/dm [deg$^{-2}\,$magAB$^{-1}$]')
ax.set_xlabel('r [magAB]')
ax.legend(fontsize=8, loc=2)

plt.show()

In [ ]:
# from minijpas_LF_and_puricomp import add_errors

# pm_flx, pm_err = add_errors(pm_flx_line, apply_err=True,
#                             survey_name='minijpasAEGIS001')

# from load_mocks import load_GAL_mock

# name_gal = f'GAL_LC_lines_0'
# pm_flx, pm_err, EW_gal, gal_zspec, gal_L, R_ang, src_list = load_GAL_mock(name_gal, return_src_list=True)

# pm_flx, pm_err = add_errors(pm_flx_line, apply_err=True,
#                             survey_name='minijpasAEGIS001')

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

bins = np.linspace(40, 43, 40)
bins_w = bins[1] - bins[0]
z_bins = [0.0, 0.7]
for i in range(len(z_bins) - 1):
        z_min = z_bins[i]
        z_max = z_bins[i + 1]
        vol = z_volume(z_min, z_max, area_obs) # * cosmo.h ** 3
        to_hist = L_OII[(gal_z > z_min) & (gal_z <= z_max)] # - 2 * np.log10(cosmo.h)
        ax.hist(to_hist, bins, histtype='step',
                weights=np.full(to_hist.shape, (vol * bins_w) ** -1),
                label=f'z={z_min}-{z_max}', color=f'C{i}')
        to_hist = L_OII_nodust[(gal_z > z_min) & (gal_z < z_max)] # - 2 * np.log10(cosmo.h)
        ax.hist(to_hist, bins, histtype='step',
                weights=np.full(to_hist.shape, (vol * bins_w) ** -1),
                linestyle='--', color=f'C{i}')
ax.set_yscale('log')
ax.legend(fontsize=10)

ax.set_xlim(40, 44)
ax.set_ylim(1e-6, 1e-1)
ax.set_xlabel('OII $\log L$')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5), rasterized=True)

h = ax.hist2d(ra, dec, bins=[200, 200])
print(ra.max() - ra.min())
# fig.colorbar(h[3])

plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

ax.hist(dist, np.linspace(0, 4000, 100))

plt.show()

In [ ]:
N_subplots = 10
mask_dist_bins = np.linspace(0, dist.max(), N_subplots**2 + 1)
fig, axs = plt.subplots(N_subplots, N_subplots, figsize=(10, 10),
                        sharex=True, sharey=True)

for iii, ax in enumerate(axs.flatten()):
    mask = (dist > mask_dist_bins[iii]) & (dist < mask_dist_bins[iii + 1])
    ax.plot(ra[mask], dec[mask], ls='', marker='.', markersize=1)
    ax.tick_params(length=0)

fig.subplots_adjust(hspace=0.01, wspace=0.01)

plt.show()

In [ ]:
N_subplots = 10
mask_dist_bins = np.linspace(dec.min(), dec.max(), N_subplots + 1)

fig, axs = plt.subplots(N_subplots, figsize=(10, 10),
                        sharex=True, sharey=True)

for iii, ax in enumerate(axs.flatten()):
    mask = (dec > mask_dist_bins[iii]) & (dec < mask_dist_bins[iii + 1])
    ax.plot(dist[mask], ra[mask], ls='', marker='.', markersize=1)
    ax.tick_params(length=0)

fig.subplots_adjust(hspace=0.01)
ax.set_xlim(0, 1500)
ax.set_xlabel('distance (Mpc)')

plt.show()